In [19]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [20]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/german_dataset/german_dataset.tsv',sep='\t', encoding="utf-8")
#balancing training data by redundancy
d1=df_train[df_train.task_1=='HOF']
print(len(d1))
d2=df_train[df_train.task_1=='NOT']
print(len(d2))
d1=pd.concat([d1,d1,d1,d1,d1,d1],ignore_index=True)
print(len(d1))
df_train=pd.concat([d1,d2],ignore_index=True)

407
3412
2442


In [21]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_1'],random_state=140) 

In [22]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [23]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/german_dataset/hasoc_de_test_gold.tsv',sep='\t', encoding="utf-8") 
test_y = encoder.fit_transform(df_test['task_1']) #encoding the label of testing data

In [24]:
# create a tfidf vectorizer object 
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=25999)
tfidf_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_tfidf =  tfidf_vect.transform(train_x)  #get tfidf vector features for training data
xvalid_tfidf =  tfidf_vect.transform(valid_x)  #get tfidf vector features for validation data
x_tfidf =  tfidf_vect.transform(df_test['text'])  #get tfidf vector features for testing data

In [25]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [26]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc(),x_tfidf.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.63      0.77      0.70       477
           1       0.88      0.78      0.83       987

    accuracy                           0.78      1464
   macro avg       0.76      0.78      0.76      1464
weighted avg       0.80      0.78      0.78      1464

classification report for testing
              precision    recall  f1-score   support

           0       0.39      0.24      0.30       220
           1       0.77      0.87      0.81       630

    accuracy                           0.71       850
   macro avg       0.58      0.55      0.56       850
weighted avg       0.67      0.71      0.68       850

